# Vogue Archive Data Processing

This notebook processes Vogue magazine data and creates vector embeddings for semantic search.

**Run this in Google Colab for free GPU access**

Runtime: ~20 minutes for 10k records

## 1. Install Dependencies

In [11]:
!pip install sentence-transformers pinecone pandas tqdm pyarrow torch transformers ftfy regex

## 2. Setup Pinecone

In [12]:
import os
from pinecone import Pinecone, ServerlessSpec

# Your Pinecone API key
PINECONE_API_KEY = "pcsk_2JKS4Y_LNuT72kmgxsuWksy2LyqcQP5Q2iX626vPCwb2KEjj23Vf72a43ZWgNp6FcCJshz"
INDEX_NAME = "vogue-archive-clip"  # New index name for CLIP embeddings

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Create index if it doesn't exist
# CLIP uses 512 dimensions (vs 384 for MiniLM)
if INDEX_NAME not in pc.list_indexes().names():
    pc.create_index(
        name=INDEX_NAME,
        dimension=512,  # CLIP ViT-B/32 dimension
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

index = pc.Index(INDEX_NAME)
print(f"Index '{INDEX_NAME}' ready!")

Index 'vogue-archive-clip' ready!


## 3. Download Pre-computed Image Embeddings

The dataset includes pre-computed CLIP image embeddings - we'll use these!

In [13]:
import numpy as np
import requests
from tqdm import tqdm

# Define the base URL for the archive
ARCHIVE_BASE_URL = "https://archive.org/download/VogueRunway_dataset"

print("Downloading pre-computed CLIP image embeddings (1.2GB - this will take a few minutes)...")
embeddings_url = f"{ARCHIVE_BASE_URL}/img_emb/VogueRunway_image.npy"

# Download with progress bar
response = requests.get(embeddings_url, stream=True)
total_size = int(response.headers.get('content-length', 0))

with open('VogueRunway_image.npy', 'wb') as f, tqdm(
    desc="VogueRunway_image.npy",
    total=total_size,
    unit='B',
    unit_scale=True,
    unit_divisor=1024,
) as pbar:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)
            pbar.update(len(chunk))

print("\nLoading embeddings into memory...")
# Load the embeddings
image_embeddings = np.load('VogueRunway_image.npy')
print(f"✓ Loaded {len(image_embeddings):,} pre-computed CLIP image embeddings")
print(f"Embedding dimension: {image_embeddings.shape[1]}")

VogueRunway_image.npy: 100%|██████████| 1.22G/1.22G [00:31<00:00, 41.9MB/s]



Loading embeddings into memory...
✓ Loaded 1,281,633 pre-computed CLIP image embeddings
Embedding dimension: 512


## 4. Load Vogue Runway Metadata

Download metadata and match with embeddings

In [14]:
import json
import pandas as pd

# Define the base URL for the archive
ARCHIVE_BASE_URL = "https://archive.org/download/VogueRunway_dataset"

print("Downloading Vogue Runway metadata...")
url = f"{ARCHIVE_BASE_URL}/VogueRunway.parquet"

# Download parquet file
response = requests.get(url, stream=True)
with open('VogueRunway.parquet', 'wb') as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

print("Loading metadata...")
df = pd.read_parquet('VogueRunway.parquet')

print(f"Total items: {len(df):,}")
print(f"Total embeddings: {len(image_embeddings):,}")

# Take top items by aesthetic score (you can change 1000 to 10000 or 100000)
NUM_ITEMS = 1000000

if 'aesthetic' in df.columns:
    df = df.nlargest(NUM_ITEMS, 'aesthetic')
    print(f"\nSelected top {NUM_ITEMS} items by aesthetic score")
else:
    df = df.head(NUM_ITEMS)
    print(f"\nTaking first {NUM_ITEMS} items")

# Reset index to get clean indices
df = df.reset_index(drop=True)

print(f"\nSample data:")
print(df[['key', 'designer', 'season', 'year', 'category', 'city']].head())

Loading metadata...
Total items: 1,281,633
Total embeddings: 1,281,633

Selected top 1000000 items by aesthetic score

Sample data:
       key  designer  season  year  category   city
0  0417267    Brioni  Spring  2019  Menswear   None
1  0673432     Cinoh  Spring  2022      None  Tokyo
2  0940024  Belstaff    Fall  2015  Menswear   None
3  1088552  Belstaff    Fall  2015  Menswear   None
4  0395237     Prada    Fall  2022  Menswear   None


## 5. Match Embeddings with Metadata and Upload to Pinecone

Use pre-computed image embeddings for multimodal search

In [ ]:
from tqdm import tqdm

# Batch size for uploading
BATCH_SIZE = 100

def process_batch(batch_df, batch_start_idx):
    """Process a batch of records with pre-computed embeddings"""
    vectors = []

    for local_idx, (_, row) in enumerate(batch_df.iterrows()):
        # Use the key to get the correct embedding
        key = int(row['key'])

        # Get embedding using the key as index
        # The embeddings array is indexed by key value
        if key < len(image_embeddings):
            embedding = image_embeddings[key].tolist()
        else:
            print(f"Warning: key {key} out of bounds, skipping")
            continue

        # Prepare metadata
        metadata = {
            "description": f"{row.get('designer', '')} {row.get('season', '')} {row.get('year', '')} {row.get('category', '')} {row.get('section', '')}".strip(),
            "designer": str(row.get('designer', '')),
            "season": str(row.get('season', '')),
            "year": int(row.get('year', 0)) if pd.notna(row.get('year')) else 0,
            "category": str(row.get('category', '')),
            "city": str(row.get('city', '')),
            "section": str(row.get('section', '')),
            "image_url": row.get('url', ''),
            "aesthetic_score": float(row.get('aesthetic', 0)) if pd.notna(row.get('aesthetic')) else 0,
        }

        vectors.append({
            "id": f"vogue_runway_{row['key']}",
            "values": embedding,
            "metadata": metadata
        })

    # Upload to Pinecone
    if vectors:
        index.upsert(vectors=vectors)
    return len(vectors)

print(f"\nProcessing {len(df)} items with pre-computed embeddings...")
print(f"Using embeddings indexed by key value")
print(f"Uploading in batches of {BATCH_SIZE}...\n")

total_uploaded = 0

for i in tqdm(range(0, len(df), BATCH_SIZE)):
    batch_df = df.iloc[i:i+BATCH_SIZE]
    count = process_batch(batch_df, i)
    total_uploaded += count

print(f"\n✓ Upload complete! {total_uploaded} vectors uploaded to Pinecone.")
print(f"\nIndex stats: {index.describe_index_stats()}")


Processing 1000000 items with pre-computed embeddings...
Using embeddings indexed by key value
Uploading in batches of 100...



  8%|▊         | 830/10000 [05:04<55:48,  2.74it/s]

## 6. Test Multimodal Search

Test text queries against image embeddings - this is CLIP's superpower!

In [10]:
from sentence_transformers import SentenceTransformer

# Load CLIP text encoder for queries
model = SentenceTransformer('clip-ViT-B-32')

# Test queries
test_queries = [
    "elegant evening gown",
    "minimalist black dress",
    "tweed jacket",
    "vintage cocktail dress"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: '{query}'")
    print(f"{'='*60}")

    # Encode text query with CLIP
    query_embedding = model.encode(query).tolist()

    # Search against image embeddings
    results = index.query(
        vector=query_embedding,
        top_k=3,
        include_metadata=True
    )

    for i, match in enumerate(results['matches'], 1):
        print(f"\n{i}. Score: {match['score']:.3f}")
        print(f"   Designer: {match['metadata']['designer']}")
        print(f"   {match['metadata']['season']} {match['metadata']['year']}")
        print(f"   Category: {match['metadata']['category']}")
        print(f"   City: {match['metadata']['city']}")
        if match['metadata'].get('image_url'):
            print(f"   Image: {match['metadata']['image_url'][:80]}...")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

0_CLIPModel/pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

0_CLIPModel/model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



Query: 'elegant evening gown'

1. Score: 0.336
   Designer: Marchesa
   Pre-Fall 2016.0
   Category: None
   City: None
   Image: https://assets.vogue.com/photos/56717da356c0c7ad4ef94587/marchesa-pre-fall-2016-...

2. Score: 0.332
   Designer: Reem Acra
   Spring 2022.0
   Category: Ready-to-Wear
   City: None
   Image: https://assets.vogue.com/photos/612e9162356f3643e41f6111/00014-ReemAcra-Spring-R...

3. Score: 0.332
   Designer: Pamella Roland
   Spring 2021.0
   Category: Ready-to-Wear
   City: None
   Image: https://assets.vogue.com/photos/5f5df5f9ad4a6163373ab00a/00020-Pamella-Roland-RT...

Query: 'minimalist black dress'

1. Score: 0.321
   Designer: Haney
   Resort 2016.0
   Category: None
   City: None
   Image: https://assets.vogue.com/photos/5756dd58e1e977d92b7e9c26/07-haney-resort-17.jpg...

2. Score: 0.319
   Designer: Redemption
   Resort 2017.0
   Category: None
   City: None
   Image: https://assets.vogue.com/photos/593accfc5ec8e2084ccc20b4/19-redemption-resort-20...



## Done! Multimodal Search Ready

Your Vogue archive now uses **image embeddings** in the database.

When users search with text, CLIP matches:
- Text query → Image embeddings
- This finds visually similar runway looks based on semantic understanding

**Benefits:**
✓ Faster processing (no embedding generation needed)
✓ Better visual understanding (searches actual image features)
✓ True multimodal CLIP search (text-to-image matching)

Next steps:
1. Deploy the API (see ../api/)
2. Your React Native app is already configured!